In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add_aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio

In [3]:
;; initialise with current date and initial capital (= $10000)
(init_portfolio "1980-12-16" 10000);

### Write a strategy

The following code implements a simple trading strategy:

In a timespan of 10 days (inclusive of today),
- Buy 50 stocks of AAPL on the first day
- Sell 10 stocks of AAPL on every other day

In [4]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days)
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days)
            (do
                (order "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get_date)))
            )
        )
        (if (odd? @num-of-days)
            (do
                (order "AAPL" -10) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get_date)))
            )
        )
        
        (update-eval-report (get_date))
        
        ; move on to the next trading day
        (next_date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

; check whether counter == 0
(println ((fn [counter] (str "Counter: " counter)) @num-of-days))

26.093628318359524
Buy 50 stocks of AAPL on 1980-12-16
26.4186732315392
Sell 10 stocks of AAPL on 1980-12-17
27.669667928008312
Sell 10 stocks of AAPL on 1980-12-19
28.803032950478386
Sell 10 stocks of AAPL on 1980-12-23
30.10884791386528
Sell 10 stocks of AAPL on 1980-12-26
29.41870999204002
Sell 10 stocks of AAPL on 1980-12-30
Counter: 0


nil

### Check order record

In [5]:
(pprint/print-table (deref order_record))


|      :date | :tic |  :price | :quantity | :total | :reference |
|------------+------+---------+-----------+--------+------------|
| 1980-12-17 | AAPL | 25.9375 |        50 |     50 |          2 |
| 1980-12-18 | AAPL | 26.6875 |       -10 |     40 |          3 |
| 1980-12-22 | AAPL | 29.6875 |       -10 |     30 |          5 |
| 1980-12-24 | AAPL | 32.5625 |       -10 |     20 |          7 |
| 1980-12-29 | AAPL | 36.0625 |       -10 |     10 |          9 |
| 1980-12-31 | AAPL | 34.1875 |       -10 |      0 |         11 |


nil

### Check portfolio record

In [6]:
;; view final portfolio
(println (deref portfolio)) ;; display it in a table 

{:cash {:tot_val 10295.0}, AAPL {:price 34.1875, :aprc 29.41870999204002, :quantity 0, :tot_val 0.0}}


nil

In [7]:
;; view portfolio value and return
(pprint/print-table (deref portfolio_value)) 

;; round to the nearest dollar
;; multiply daily-ret by 100 


|      :date |         :tot_value |            :daily_ret |
|------------+--------------------+-----------------------|
| 1980-12-16 |              10000 |                     0 |
| 1980-12-17 | 10007.806415917976 |  7.803370496319297E-4 |
| 1980-12-18 | 10026.746929261568 | 0.0018907852508677928 |
| 1980-12-22 |  10096.96503784025 |  0.006978672093164625 |
| 1980-12-24 | 10168.560659009569 |  0.007065784535722363 |
| 1980-12-29 | 10254.213479138652 |  0.008388020329870642 |
| 1980-12-31 |            10295.0 |  0.003969648226449433 |


nil

### Generate evaluation report

In [10]:
(eval-report)


|      :date | :pnl-per-trade | :annualised-sharpe | :annualised-ret | :volatility | :total-val | :total-ret | :daily-ret | :annualised-vol | :sharpe |
|------------+----------------+--------------------+-----------------+-------------+------------+------------+------------+-----------------+---------|
| 1980-12-16 |           7.81 |              0.001 |             0.0 |       0.001 |   10007.81 |      0.001 |      0.001 |           0.009 |   1.414 |
| 1980-12-17 |          13.37 |              0.001 |             0.0 |       0.001 |   10026.75 |      0.003 |      0.002 |           0.015 |   2.811 |
| 1980-12-18 |          13.37 |              0.001 |             0.0 |       0.001 |   10026.75 |      0.003 |      0.002 |           0.015 |   2.811 |
| 1980-12-19 |          32.32 |              0.001 |             0.0 |       0.003 |   10096.97 |       0.01 |      0.007 |            0.05 |   3.072 |
| 1980-12-22 |          32.32 |              0.001 |             0.0 |       0.003 |   

nil

### Plot variables

In [ ]:
;; to-do